<center>
    <h1> Scientific Programming in Python  </h1>
    <h2> Topic 4: Just in Time Compilation: Numba and NumExpr </h2> 
</center>

_Notebook created by Felipe Mancilla S - `felipe.mancilla@alumnos.usm.cl` - DI UTFSM - April 2017._

In [1]:
import numba
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt
import scipy as sp

En esta actividad implementaremos una conocida métrica para medir disimilitud entre conjuntos: __La métrica de Hausdorff__. Esta corresponde a un métrica o distancia ocupada para medir cuán disímiles son dos subconjuntos dados. 


Esta tiene muchas aplicaciones, en particular para comparar el parecido entre imágenes. En el caso en donde los conjuntos son arreglos bidimensionales, la definición es la siguiente:

Sean $X \in \mathbb{R}^{m \times 3}$ e  $Y \in \mathbb{R}^{n \times 3}$ dos matrices, la métrica/distancia de Hausdorff sobre sobre estas como:

$$
d_H(X,Y) = \max \left(\ \max_{i\leq m} \min_{j \leq n} d(X[i],Y[j]), \ \max_{j\leq n} \min_{i \leq m} d(Y[j],X[i]) \ \right)
$$

donde $d$ es la _distancia Euclideana_ clásica. ($X[i]$ indíca la i-ésima fila de X).

__Ilustración unidimensional:__ Distancia entre funciones.
<img src='data/hausdorff.png' style="width: 600px;">

1. Implemente la métrica de Hausdorff en Python clásico.
2. Implemente la métrica de Hausdorff usando Numba (Forzando el modo __nopython__ y definiendo explícitamente las _signatures_ de las funciones).
3. Cree `10` arreglos $X,Y$ aleatorios, con cantidad creciente de filas, y realice análsis de tiempos de ejecuciones de las funciones anteriores sobre estos arreglos.
4. Concluya.

1. Implemente la métrica de Hausdorff en Python clásico.

Para esta parte se implementara esta métrica de dos formas, con una función de la libreria scipy.spatial llamado "distance"

In [2]:
from scipy.spatial import distance
def metric_Hausdorff(X,Y):
    dst_x = np.zeros(shape=(len(X),len(Y)))
    dst_y = np.zeros(shape=(len(Y),len(X)))
    minimum_x = np.zeros(len(X))
    minimum_y = np.zeros(len(Y))
    for i in range(0,len(X)):
        for j in range(0,len(Y)):
            dst_x[i,j]=distance.euclidean(X[i,:],Y[j,:])
        minimum_x[i]=min(dst_x[i,:])
    for i in range(0,len(Y)):
        for j in range(0,len(X)):
            dst_y[i,j]=distance.euclidean(Y[i,:],X[j,:])
        minimum_y[i]=min(dst_y[i,:])
    return max(max(minimum_x),max(minimum_y))

Y de la otra forma, usando funciones de la libreria "numpy"

In [3]:
def metric_Hausdorff_numpy(X,Y):
    dst_x = np.zeros(shape=(len(X),len(Y)))
    dst_y = np.zeros(shape=(len(Y),len(X)))
    minimum_x = np.zeros(len(X))
    minimum_y = np.zeros(len(Y))
    for i in range(0,len(X)):
        for j in range(0,len(Y)):
            dst_x[i,j]=np.sqrt(np.sum((X[i,:]-Y[j,:])**2))
        minimum_x[i]=dst_x[i,:].min()
    for i in range(0,len(Y)):
        for j in range(0,len(X)):
            dst_y[i,j]=np.sqrt(np.sum((Y[i,:]-X[j,:])**2))
        minimum_y[i]=dst_y[i,:].min()
    return max(minimum_x.max(),minimum_y.max())

Por lo tanto, sacaremos los tiempos de cada función.

In [4]:
maximum = np.zeros(10)
for i in range(3,13):
   
    #print(i,i-3)
    X = np.zeros(shape=(i,3))
    ad=np.random.randint(0,2)
    Y = np.zeros(shape=(i+ad,3))
    X= np.random.random((i,3))
    Y= np.random.random((i+ad,3))
    
    %%timeit maximum[i-3] = metric_Hausdorff(X,Y)

The slowest run took 376.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 369 µs per loop
1000 loops, best of 3: 657 µs per loop
1000 loops, best of 3: 1 ms per loop
1000 loops, best of 3: 1.49 ms per loop
100 loops, best of 3: 1.92 ms per loop
100 loops, best of 3: 2.84 ms per loop
100 loops, best of 3: 3.64 ms per loop
100 loops, best of 3: 4.52 ms per loop
100 loops, best of 3: 4.76 ms per loop
100 loops, best of 3: 6.28 ms per loop


In [5]:
maximum = np.zeros(10)
for i in range(3,13):
   
    #print(i,i-3)
    X = np.zeros(shape=(i,3))
    ad=np.random.randint(0,2)
    Y = np.zeros(shape=(i+ad,3))
    X= np.random.random((i,3))
    Y= np.random.random((i+ad,3))
    
    %%timeit maximum[i-3] = metric_Hausdorff_numpy(X,Y)

1000 loops, best of 3: 221 µs per loop
1000 loops, best of 3: 333 µs per loop
1000 loops, best of 3: 500 µs per loop
1000 loops, best of 3: 604 µs per loop
1000 loops, best of 3: 838 µs per loop
1000 loops, best of 3: 1.01 ms per loop
1000 loops, best of 3: 1.42 ms per loop
1000 loops, best of 3: 1.68 ms per loop
100 loops, best of 3: 1.98 ms per loop
100 loops, best of 3: 2.36 ms per loop


Podemos observar que nuestra segunda función usando numpy es mas rapida que la función usada de la libreria scipy. 
Ahora implementando la función en __Numba__.

In [12]:
from scipy.spatial import distance
@numba.jit('float64 (float64[:,:], float64[:,:])',nopython=True)
def metric_Hausdorff_numba(X,Y):
    dst_x = np.zeros(shape=(len(X),len(Y)))
    dst_y = np.zeros(shape=(len(Y),len(X)))
    minimum_x = np.zeros(len(X))
    minimum_y = np.zeros(len(Y))
    for i in range(0,len(X)):
        for j in range(0,len(Y)):
            dst_x[i,j]=np.sqrt(np.sum((X[i,:]-Y[j,:])**2))
        minimum_x[i]=dst_x[i,:].min()
    for i in range(0,len(Y)):
        for j in range(0,len(X)):
            dst_y[i,j]=np.sqrt(np.sum((Y[i,:]-X[j,:])**2))
        minimum_y[i]=dst_y[i,:].min()
    return max(minimum_x.max(),minimum_y.max())

Entonces al observar su tiempo de ejecución

In [15]:
maximum = np.zeros(10)
for i in range(3,13):
    #print(i,i-3)
    X = np.zeros(shape=(i,3))
    ad=np.random.randint(0,2)
    Y = np.zeros(shape=(i+ad,3))
    X= np.random.random((i,3))
    Y= np.random.random((i+ad,3))
    %%timeit maximum[i-3] = metric_Hausdorff_numba(X,Y)

The slowest run took 4.70 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 6.04 µs per loop
100000 loops, best of 3: 9.04 µs per loop
100000 loops, best of 3: 11.2 µs per loop
100000 loops, best of 3: 17.3 µs per loop
10000 loops, best of 3: 22.6 µs per loop
10000 loops, best of 3: 28.5 µs per loop
10000 loops, best of 3: 35 µs per loop
10000 loops, best of 3: 38.9 µs per loop
10000 loops, best of 3: 46.1 µs per loop
10000 loops, best of 3: 55.9 µs per loop


En conclusión, analizando los tiempos de ejecución de cada funcion implementada en __scypy__,__numpy__ y __numba__.
Se concluye que el mejor tiempo ocurre en __numba__, siendo que el peor caso de la cantidad de arreglos (10x3) aun se calcula la métrica de manera "demasiado" rapido.